In [ ]:
!pip install -r requirements.txt

In [2]:
# Importa SparkSession
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[1]").appName("lucasAppEleflow").config("spark.sql.catalogImplementation","hive").enableHiveSupport().getOrCreate()

In [3]:
# !pip freeze >> requirements.txt

In [5]:
os.getcwd()

'/home/jovyan/work/src'

In [4]:
# Importação de bibliotecas 
import os 
import json 
import re
from pyspark.sql.functions import col
import unidecode
import pyspark.sql.functions

In [4]:
# Pega caminho do diretório para trabalho 
from utils import get_project_root

root_dir = get_project_root()
print(root_dir)

/home/jovyan/work


In [5]:
# Caminho da pasta /VRA
path_VRA = os.path.join(root_dir, 'VRA') 
path_AIR_CIA = os.path.join(root_dir, 'AIR_CIA')
path_AIR_CIA

'/home/jovyan/work/AIR_CIA'

In [6]:
# Lista de todos os arquivos .json da pasta /VRA
list_files_VRA = [f for f in os.listdir(path_VRA) if len(re.findall('.json',f))!=0]
list_files_AIR_CIA = [f for f in os.listdir(path_AIR_CIA) if len(re.findall('.csv',f))!=0]
list_files_AIR_CIA

['ANAC_20211220_203627.csv',
 'ANAC_20211220_203643.csv',
 'ANAC_20211220_203733.csv',
 '.~lock.ANAC_20211220_203733.csv#',
 'airport_api_data.csv']

In [7]:
# Função que converte strings em CamelCase para snake_case
def camel_to_snake(name):
    name = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    name = unidecode.unidecode(name)
    return re.sub('([a-z0-9])([A-Z])', r'\1_\2', name).lower()

In [8]:
# Função que converte (e salva) cada um dos arquivos .json da pasta /VRA para terem cabeçalho em snake_case
def convert_header_to_snake_case_VRA(i):
    try: 
        df = spark.read.json(path_VRA +'/'+ list_files_VRA[i])
        mapping = dict([[c,camel_to_snake(c)] for c in df.columns])
        df = df.select([col(c).alias(mapping.get(c, c)) for c in df.columns])
        df.toPandas().to_json(path_VRA+'/'+list_files_VRA[i], orient='records', force_ascii=False, lines=True)
        del df
        print(r"O arquivo {file} da pasta /VRA agora tem cabeçalho em snake_case!".format(file=list_files_VRA[i]))
    except Exception as e:
        print(e)

In [9]:
# Normaliza e salva todos arquivos .json da pasta /VRA para terem cabeçalho em snake_case
[convert_header_to_snake_case_VRA(j) for j in range(len(list_files_VRA))]

O arquivo VRA_20211.json da pasta /VRA agora tem cabeçalho em snake_case!
O arquivo VRA_202110.json da pasta /VRA agora tem cabeçalho em snake_case!
O arquivo VRA_202111.json da pasta /VRA agora tem cabeçalho em snake_case!
O arquivo VRA_20212.json da pasta /VRA agora tem cabeçalho em snake_case!
O arquivo VRA_20213.json da pasta /VRA agora tem cabeçalho em snake_case!
O arquivo VRA_20214.json da pasta /VRA agora tem cabeçalho em snake_case!
O arquivo VRA_20215.json da pasta /VRA agora tem cabeçalho em snake_case!
O arquivo VRA_20216.json da pasta /VRA agora tem cabeçalho em snake_case!
O arquivo VRA_20217.json da pasta /VRA agora tem cabeçalho em snake_case!
O arquivo VRA_20218.json da pasta /VRA agora tem cabeçalho em snake_case!
O arquivo VRA_20219.json da pasta /VRA agora tem cabeçalho em snake_case!


[None, None, None, None, None, None, None, None, None, None, None]

In [10]:
# re.sub(' ',r'_',lc[0]).lower()
# Função que converte strings em CamelCase para snake_case
def upper_to_snake(name):
    name = re.sub(' ',r'_',name)
    name = unidecode.unidecode(name)
    name = re.sub('(.)([A-Z][a-z]+)', r'\1\2', name).lower()
    return name 
upper_to_snake('Razão e-mail')

'razao_e-mail'

In [11]:
# Função que converte (e salva) cada um dos arquivos .csv da pasta /VRA para terem cabeçalho em snake_case
def convert_header_to_snake_case_AIR_CIA(i):
    try: 
        df = spark.read.options(delimiter=';', header='True').csv(path_AIR_CIA +'/'+ list_files_AIR_CIA[i])
        mapping = dict([[c,upper_to_snake(c)] for c in df.columns])
        df = df.select([col(c).alias(mapping.get(c, c)) for c in df.columns])
        split_col = pyspark.sql.functions.split(df['icao_iata'], r' ')
        df = df.withColumn('icao', split_col.getItem(0)) \
               .withColumn('iata', split_col.getItem(1))
        df.show(truncate=False)
        df.toPandas().to_csv(path_AIR_CIA+'/'+list_files_AIR_CIA[i])
        # df.printSchema()
        # df.show()
        del df
        print(r"O arquivo {file} da pasta /AIR_CIA agora tem cabeçalho em snake_case!".format(file=list_files_AIR_CIA[i]))
    except Exception as e:
        print(e)

In [12]:
# Normaliza e salva todos arquivos .csv da pasta /AIR_CIA para terem cabeçalho em snake_case
[convert_header_to_snake_case_AIR_CIA(j) for j in range(len(list_files_AIR_CIA))]

Cannot resolve column name "icao_iata" among (,razao_social,icao_iata,cnpj,atividades_aereas,endereco_sede,telefone,e-mail,decisao_operacional,data_decisao_operacional,validade_operacional,icao,iata)
Cannot resolve column name "icao_iata" among (,razao_social,icao_iata,cnpj,atividades_aereas,endereco_sede,telefone,e-mail,decisao_operacional,data_decisao_operacional,validade_operacional,icao,iata)
Cannot resolve column name "icao_iata" among (,razao_social,icao_iata,cnpj,atividades_aereas,endereco_sede,telefone,e-mail,decisao_operacional,data_decisao_operacional,validade_operacional,icao,iata)
Unable to infer schema for CSV. It must be specified manually.
Cannot resolve column name "icao_iata" among (id,iata,icao,name,location,street_number,street,city,county,state,country_iso,country,postal_code,phone,latitude,longitude,uct,website,error)


[None, None, None, None, None]

In [13]:
dft = spark.read.options(delimiter=',', header='True').csv(path_AIR_CIA +'/'+ list_files_AIR_CIA[0])
dfp = dft.toPandas()
dfp#['icao']

,_c0,razao_social,icao_iata,cnpj,atividades_aereas,endereco_sede,telefone,e-mail,decisao_operacional,data_decisao_operacional,validade_operacional,icao,iata
0,0,ABSA - AEROLINHAS BRASILEIRAS S.A.,LTG M3,00.074.635/0001-33,TRANSPORTE AÉREO REGULAR,"AEROPORTO INTERNACIONAL DE VIRACOPOS, RODOVIA ...",(11) 5582-8055,gar@tam.com.br,DECISÃO Nº 41,22/04/2015,23/04/2025,LTG,M3
1,1,AEROSUL TÁXI AÉREO LTDA (EX.: AUSTEN TÁXI AÉRE...,ASO 2S,27.315.694/0001-02,"SERVIÇOS AÉREOS PÚBLICOS, TÁXI-AÉREO, TRANSPOR...","RODOVIA PR 218, KM 7, JARDIM UNIVERSITÁRIO, AE...",(43) 3176-4030,operacoes@aerosul.com.br,DECISÃO Nº 282,10/02/2021,None,ASO,2S
2,2,ASTA LINHAS AÉREAS LTDA ( EX - AMÉRICA DO SUL ...,SUL 0A,12.703.737/0001-50,"SERVIÇOS AÉREOS PÚBLICOS, TRANSPORTE AÉREO REG...","AV GOVERNADOR JOÃO PONCE DE ARRUDA, S/N, HANGA...",(65) 3614-2684 / (18) 2104-1000,None,DECISÃO Nº 115,31/07/2019,02/08/2024,SUL,0A
3,3,ATA - AEROTÁXI ABAETÉ LTDA.,ABJ,14.674.451/0001-19,"TÁXI-AÉREO, TRANSPORTE AÉREO NÃO REGULAR, TRAN...","PÇA. GAGO COUTINHO, S/N, AEROPORTO INTERNACIO...",(71) 34629600 / (71) 996643563,guilherme@abaete.com.br,DECISÃO Nº 36,05/03/2020,None,ABJ,None
4,4,AZUL CONECTA LTDA. (EX TWO TAXI AEREO LTDA),ACN 2F,04.263.318/0001-16,TRANSPORTE AÉREO REGULAR,"AVENIDA EMILIO ANTONON, 901, BAIRRO CHÁCARA AE...",(11) 45822355/ (11) 987433988,None,"Decisão nº 14, de 25/01/2018",25/01/2018,29/01/2028,ACN,2F
5,5,AZUL LINHAS AÉREAS BRASILEIRAS S/A,AZU AD,09.296.295/0001-60,TRANSPORTE AÉREO REGULAR,"AV. MARCOS PENTEADO ULHÔA RODRIGUES,N°939, 9° ...",(55) (11) 4134-9887 | Fax: (55)(11)4134-9890,vanessa.reis@voeazul.com.br,Decisão nº 178,20/11/2018,21/11/2028,AZU,AD
6,6,GOL LINHAS AÉREAS S.A. (EX- VRG LINHAS AÉREAS ...,GLO G3,07.575.651/0001-59,TRANSPORTE AÉREO REGULAR,"PRAÇA SENADOR SALGADO FILHO, S/N°, AEROPORTO S...",(11) 5098-2990 / (11) 5098-7872,rddfonseca@voegol.com.br,DECISÃO Nº 156,05/11/2019,None,GLO,G3
7,7,MAP TRANSPORTES AÉREOS LTDA.,PAM 7M,10.483.635/0001-40,TRANSPORTE AÉREO REGULAR,"AVENIDA SANTOS DUMONT, Nº 1350, AEROPORTO INTE...",(92) 36521620,contabil@voemap.com.br,DECISÃO Nº 89,14/08/2012,17/08/2022,PAM,7M
8,8,MODERN TRANSPORTE AEREO DE CARGA S.A,MWM WD,03.887.831/0001-15,TRANSPORTE AÉREO REGULAR,"AVENIDA PREFEITO LUIZ LATORRE, Nº 9450, CONDOM...",(21) 2272-9272 / (11) 31096750,juridico@modern.com.br,DECISÃO Nº 79,31/05/2017,02/06/2027,MWM,WD
9,9,PASSAREDO TRANSPORTES AÉREOS S.A.,PTB 2Z,00.512.777/0001-35,TRANSPORTE AÉREO REGULAR,SETOR DE HANGARES. ENTRADA PELA AV. THOMAZ ALB...,(16) 3514-7127 / (16) 3514-7126,juridico@voepass.com.br,DECISÃO Nº 105,09/06/2020,None,PTB,2Z


In [14]:
list_icao = list(dfp['icao']) # 'icao' , 'iata'
list_icao

['LTG',
 'ASO',
 'SUL',
 'ABJ',
 'ACN',
 'AZU',
 'GLO',
 'PAM',
 'MWM',
 'PTB',
 'SID',
 'TAM',
 'TTL']

In [15]:
from functools import reduce
from pyspark.sql import DataFrame

SeriesAppend=[]
for i in range(len(list_files_VRA)):
    temp = spark.read.json(path_VRA +'/'+ list_files_VRA[i])
    SeriesAppend.append(temp)
df_series = reduce(DataFrame.unionAll, SeriesAppend)
# df_series.toPandas()

In [16]:
selected = [s for s in df_series.columns if len(re.findall('icao', s))!=0]
df_icao = df_series.select(selected).distinct()

In [17]:
def get_unique_icao(i):
    temp = set(df_icao.select(df_icao.columns[1]).distinct().toPandas().values.flatten())
    return temp

list_icao = set({})
for i in range(len(df_icao.columns)):
    temp = get_unique_icao(i)
    list_icao = list_icao.union(temp)
# list_icao
list_icao = list(list_icao)

In [18]:
from pyspark.sql import SparkSession
import requests
import json
from pyspark.sql.functions import udf, col, explode
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, ArrayType
from pyspark.sql import Row

In [19]:
from pyspark.sql.types import TimestampType, StringType

# Path to our 20 JSON files
inputPath = os.path.join(root_dir, 'streaming')
# inputPath = root_dir+"/streaming"
inputPath

'/home/jovyan/work/streaming'

In [20]:
import asyncio
import aiohttp
from pprint import pprint
import time

def api():
    list_res = list()
    for i in range(len(list_icao)):
        code = list_icao[i]
        headers = {
                    "X-RapidAPI-Key": "423d333f09mshb6e682cf155805fp1fcec8jsn1f4f54ab45e6",
                    "X-RapidAPI-Host": "airport-info.p.rapidapi.com"
                }

        url = "https://airport-info.p.rapidapi.com/airport"
        querystring = {"icao": str(code).upper()}
        async def fetch(url, session):
            
            async with session.get(url, headers=headers, params=querystring) as response:
                res = await response.json()
                list_res.append(res)
                pprint(list_res)
                with open(inputPath+'/api_data.json', 'a') as json_file:
                    json.dump(res, json_file)
                time.sleep(1)
                return list_res


        async def main():
            async with aiohttp.ClientSession() as session:
                await fetch(url, session)

        try:
            loop = asyncio.get_running_loop()
        except RuntimeError:  # 'RuntimeError: There is no current event loop...'
            loop = None

        if loop and loop.is_running():
            #print('Async event loop already running. Adding coroutine to the event loop.')
            tsk = loop.create_task(main())
            # ^-- https://docs.python.org/3/library/asyncio-task.html#task-object
            # Optionally, a callback function can be executed when the coroutine completes
            tsk.add_done_callback(
                lambda t: print(f'Task done with result={t.result()}  << return val of main()'))
        else:
            #print('Starting new event loop')
            asyncio.run(main())
# api()

In [21]:
import time
timeout = time.time() + 60*5   # 5 minutes from now
while True:
    api()
    if time.time() > timeout:
        break

KeyboardInterrupt: 

Criar as seguintes views (Priorize o uso de SQL para esta parte):
Para cada companhia aérea trazer a rota mais utilizada com as seguintes informações:
* Razão social da companhia aérea
* Nome Aeroporto de Origem
* ICAO do aeroporto de origem
* Estado/UF do aeroporto de origem
* * Nome do Aeroporto de Destino
* ICAO do Aeroporto de destino
* Estado/UF do aeroporto de destino

In [ ]:
df_vra = df_series
df_vra.columns

In [ ]:
# df_api.columns
df_cia = spark.read.options(delimiter=',', header='True').csv(path_AIR_CIA +'/'+ list_files_AIR_CIA[0])
df_cia.columns

In [ ]:
df_api = spark.read.json(inputPath+'/personal.json')
df_api.columns

In [ ]:
spark.sql("create database if not exists db")

In [ ]:
try:
    df_api.write.format("orc").mode("overwrite").saveAsTable("db.api") # "append" , "overwrite"
    df_cia.write.format("orc").mode("overwrite").saveAsTable("db.cia") 
    df_vra.write.format("orc").mode("overwrite").saveAsTable("db.vra") 
except Exception as e:
    print(e)

Criar as seguintes views (Priorize o uso de SQL para esta parte):
Para cada companhia aérea trazer a rota mais utilizada com as seguintes informações:
* Razão social da companhia aérea
* Nome Aeroporto de Origem
* ICAO do aeroporto de origem
* Estado/UF do aeroporto de origem
* * Nome do Aeroporto de Destino
* ICAO do Aeroporto de destino
* Estado/UF do aeroporto de destino

In [ ]:
spark.sql("select * from db.vra").toPandas().columns
# spark.sql("select * from db.cia").toPandas().columns
# spark.sql("select * from db.api").toPandas().columns

In [ ]:
df_sql1 = spark.sql("""
            select 
                d.razao_social,
                a.name as nome_aeroporto_de_origem,
                b.icao_aerodromo_origem as icao_aeroporto_de_origem,
                a.county as uf_aeroporto_de_origem,
                e.icao_aerodromo_destino as icao_aeroporto_de_destino,
                e.county as uf_aeroporto_de_destino,
                e.country_iso
            from db.api a
            left join (
                select * from db.vra
            ) b on b.icao_aerodromo_origem = a.icao 
            left join (
                select * from db.cia
            ) d on d.icao = b.icao_aerodromo_origem
            left join (
                            select *
            from db.api a1
            left join (
                select * from db.vra
            ) b1 on b1.icao_aerodromo_destino = a1.icao 
            left join (
                select * from db.cia
            ) d1 on d1.icao = b1.icao_aerodromo_destino
            ) e
        """)
df_sql1 = df_sql1.distinct()
df_sql1.toPandas()#.columns

In [ ]:
import pyspark.sql.functions as F

df_sql1.groupby(['razao_social', 'nome_aeroporto_de_origem', 'icao_aeroporto_de_origem',
       'uf_aeroporto_de_origem', 'icao_aeroporto_de_destino',
       'uf_aeroporto_de_destino']).count().sort(F.col('count').desc()).toPandas()

Para cada aeroporto trazer a companhia aérea com maior atuação no ano com as seguintes informações:
* Nome do Aeroporto
* ICAO do Aeroporto
* Razão social da Companhia Aérea
* Quantidade de Rotas à partir daquele aeroporto
* Quantidade de Rotas com destino àquele aeroporto
* Quantidade total de pousos e decolagens naquele aeroporto

In [ ]:
df_sql2 = spark.sql("""
            select 
                a.name as nome_do_aeroporto,
                b.icao_aerodromo_origem as icao_do_aeroporto,
                d.razao_social,
                e.icao_aerodromo_destino as icao_aeroporto_de_destino,
                e.icao_aerodromo_origem as icao_aeroporto_de_origem
            from db.api a
            left join (
                select * from db.vra
            ) b on b.icao_aerodromo_origem = a.icao 
            left join (
                select * from db.cia
            ) d on d.icao = b.icao_aerodromo_origem
            left join (
                            select *
            from db.api a1
            left join (
                select * from db.vra
            ) b1 on b1.icao_aerodromo_destino = a1.icao 
            left join (
                select * from db.cia
            ) d1 on d1.icao = b1.icao_aerodromo_destino
            ) e
        """)
df_sql2 = df_sql2.distinct()
df_sql2.toPandas()

In [ ]:
df_sql2.groupby(['razao_social', 'nome_do_aeroporto', 'icao_do_aeroporto',
       'razao_social', 'icao_aeroporto_de_destino',
       'icao_aeroporto_de_origem']).count().sort(F.col('count').desc()).toPandas()